In [1]:
import numpy as np
import os
import json
from imgaug import augmenters as iaa
import imgaug as ia
import skimage.io as io
import imageio
from pycocotools.coco import COCO
import pycocotools.mask as cocomask
from itertools import groupby

In [2]:
datasets = ['train', 'valid', 'test', 'difficult']
root = os.getcwd()

def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle

In [3]:
#can add different augmentation
rot = iaa.Flipud(1.0)

#old folders
old_folder_images = os.path.join(root, 'images_split')
old_folder_annotations = os.path.join(root, 'annotations_split')

#create new folders
new_folder_images = os.path.join(root, 'images')
new_folder_annotations = os.path.join(root, 'annotations')
os.makedirs(os.path.join(root, 'images'), exist_ok=True)
os.makedirs(os.path.join(root, 'annotations'), exist_ok=True)

#iterate through datasets
for dataset in datasets:
    #create new folders
    new_path_images = os.path.join(new_folder_images, dataset)
    os.makedirs(new_path_images, exist_ok=True)
    
    #annotations path
    old_path_annotations = os.path.join(old_folder_annotations, dataset + '.json')
    new_path_annotations = os.path.join(new_folder_annotations, dataset + '.json')
    
    #load data
    coco = COCO(old_path_annotations)
    with open(old_path_annotations) as f:
        data = json.load(f)
        cat_ids = coco.getCatIds()
        anns_ids = coco.getAnnIds(catIds=cat_ids, iscrowd=None)
        anns = coco.loadAnns(anns_ids)
    
    #iterate through annotations
    for item in anns:
        #read image
        image_name = coco.loadImgs(item['image_id'])[0]['file_name']
        image_path = os.path.join(old_folder_images, dataset, image_name)
        image = io.imread(image_path)
        
        #read mask
        mask = coco.annToMask(item)
        segmap = ia.SegmentationMapOnImage(mask, shape=image.shape)
        
        #read bounding box
        bb = ia.augmentables.bbs.BoundingBoxesOnImage([ia.augmentables.bbs.BoundingBox(item['bbox'][0], item['bbox'][1], 
                                                             item['bbox'][0] + item['bbox'][2],
                                                             item['bbox'][1] + item['bbox'][3])], shape=image.shape)
        
        #apply augmentation
        image_aug = rot.augment_image(image)
        segmap_aug = rot.augment_segmentation_maps(segmap)
        bb_aug = rot.augment_bounding_boxes(bb)
        
        #save augmentation
        io.imsave(os.path.join(new_path_images, image_name), image_aug)
        item['segmentation'] = binary_mask_to_rle(segmap_aug.get_arr())
        item['bbox'] = [bb_aug[0].x1, bb_aug[0].y1, bb_aug[0].width, bb_aug[0].height]
    
    #save json
    data["annotations"] = anns
    with open(new_path_annotations, 'w') as f:
        json.dump(data, f)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


C:\Users\Sveta\anaconda3\envs\CudaLab\lib\site-packages\imgaug\imgaug.py:184: DeprecationWarning: Function `SegmentationMapOnImage()` is deprecated. Use `SegmentationMapsOnImage` instead. (Note the plural 'Maps' instead of old 'Map'.).
  warn_deprecated(msg, stacklevel=3)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
